In [25]:
import random
from math import gcd

class RSA:
    # Constructor initializing RSA with default key length, character set and public and private key dictionaries
    def __init__(self, key_length=2048):
        self.key_length = key_length
        self.clear_alphabet = "abcdefghijklmnopqrstuvwxyzåäö ?!%/&.,1234567890"
        self.public_key = {}
        self.private_key = {}

    # Converts a message string to a list of integers based on a character-number mapping
    def get_number_from_message(self, message):
        char_to_num = {char: str(index + 10) for index, char in enumerate(self.clear_alphabet)}
        numbers = [char_to_num.get(char, '') for char in message]
        return [int(num) for num in numbers if num]

    # Converts a list of integers back to a message string using a character-number mapping
    def get_message_from_number(self, numbers):
        return ''.join([self.clear_alphabet[num - 10] for num in numbers])

    # Encrypts a message by converting its characters into numerical values and applying modular exponentiation with the public key's exponent 'E' and modulus 'N'
    def encrypt(self, message):
        message_numbers = self.get_number_from_message(message)
        encrypted_numbers = [pow(num, self.public_key['E'], self.public_key['N']) for num in message_numbers]
        return ' '.join(map(str, encrypted_numbers))

    # Decrypts an encrypted message by splitting it into numerical values and applying modular exponentiation with the private key's decryption exponent 'D' and modulus 'N'
    def decrypt(self, encrypted_message):
        encrypted_numbers = [int(num) for num in encrypted_message.split()]
        decrypted_numbers = [pow(num, self.private_key['D'], self.private_key['N']) for num in encrypted_numbers]
        return self.get_message_from_number(decrypted_numbers)

    # Checks whether a number 'n' is likely prime by applying the Miller-Rabin primality test after performing initial divisibility checks
    def is_prime(self, n, k=5):
        if n <= 1:
            return False
        if n <= 3:
            return True
        if n % 2 == 0:
            return False

        # Run the Miller-Rabin primality test k times
        for _ in range(k):
            a = random.randint(2, n - 2)
            x = pow(a, n - 1, n)
            if x != 1:
                return False

        return True

    # Generates random prime number within the specified length, ensuring that the number is both odd and greater than 1
    def get_random_prime(self):
        while True:
            candidate = random.randint(2**(self.key_length - 1) + 1, 2**self.key_length - 1)
            if candidate % 2 == 0:
                continue  # Ensure it's odd
            if self.is_prime(candidate):
                return candidate
            
    # Checks if two numbers a and b are relatively prime, the security of RSA encryption is based on the computational difficulty of factoring the product of two large prime numbers
    def is_relative_prime(self, a, b):
        return gcd(a, b) == 1
            
    # Calculates the modular inverse of 'a' modulo 'n' using the extended Euclidean algorithm, ensuring 'a' has an inverse modulo 'n' and returning the modular inverse if it exists
    def mod_inverse(self, a, n):
        t, newt = 0, 1
        r, newr = n, a

        while newr != 0:
            quotient = r // newr
            t, newt = newt, t - quotient * newt
            r, newr = newr, r - quotient * newr

        if r > 1:
            raise ValueError("a is not invertible")
        if t < 0:
            t += n

        # Returns the private key component necessary for secure decryption of messages
        return t
    
    def generate_key_pair(self):
        # Generate two random prime numbers
        p = self.get_random_prime()
        q = self.get_random_prime()

        # Calculate N and phi(N)
        N = p * q
        phi_N = (p - 1) * (q - 1)

        # Choose a suitable public exponent (E)
        E = 65537  # Common choice for E
        while not self.is_relative_prime(E, phi_N):
            E += 1

        # Calculate the private exponent (D)
        D = self.mod_inverse(E, phi_N)

        # Store the public and private keys
        self.public_key = {'N': N, 'E': E}
        self.private_key = {'N': N, 'D': D}

In [26]:
if __name__ == "__main__":
    rsa = RSA(key_length=2048)
    rsa.generate_key_pair()

    message = str.lower("your text")

    print(f"Generated Public Key (N, E): ({rsa.public_key['N']}, {rsa.public_key['E']}) \n")
    print(f"Generated Private Key (D): {rsa.private_key['D']} \n")

    encrypted_message = rsa.encrypt(message)
    print(f"Encrypted Message: ({encrypted_message}) \n")

    decrypted_message = rsa.decrypt(encrypted_message)
    print(f"Decrypted Message: {decrypted_message}")

Generated Public Key (N, E): (6592100793270685683979992216408607577553771107635972499799749620174370047630844177606777350923315884685856114231507772032522868290822194175072573827203438846662017047835303635209349983429515994227177857535320033338707793013996026126282930223960259736034780267854262749822609323760744076832171324442220577555431527641950666705208053591337175753853034393861193258518095981346439238292674251486317664097999401218084830909681860538352331889841156183591467268002254214022582806136404691270833967624950947401825726620345897406585853639294754128829085234598374927036949187033763648757156127491627598234857238396049003413161699367226998529971452856151270125853149702288661175001538316179897919051739471502360784569878118385313734568781061857066249536986408322967637895261988204922268918267357264910556921617875010016031223952309048913333549495818958454987254229100281710011810082277585701906890393028052888032599042175320220996331222536944722450281282903214998645497551385197249